In [12]:
# Import necessary libraries
import numpy as np
import random
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional
from copy import deepcopy
from collections import defaultdict

In [13]:
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)

    def is_valid_move(self, x: int, y: int) -> bool:
        return 0 <= x < 3 and 0 <= y < 3 and self.board[x, y] == 0

    def make_move(self, x: int, y: int, player: int) -> bool:
        if self.is_valid_move(x, y):
            self.board[x, y] = player
            return True
        return False

    def check_winner(self) -> Optional[int]:
        # Check rows, columns and diagonals
        for i in range(3):
            if abs(sum(self.board[i, :])) == 3:
                return np.sign(sum(self.board[i, :]))
            if abs(sum(self.board[:, i])) == 3:
                return np.sign(sum(self.board[:, i]))
        
        if abs(self.board.trace()) == 3:
            return np.sign(self.board.trace())
        if abs(np.fliplr(self.board).trace()) == 3:
            return np.sign(np.fliplr(self.board).trace())
        
        return None if 0 in self.board else 0

    def get_available_moves(self) -> List[Tuple[int, int]]:
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def display(self):
        display_board = self.board.astype(str)
        display_board[display_board == "0"] = "."
        display_board[display_board == "1"] = "X"
        display_board[display_board == "-1"] = "O"
        print("\n".join([" ".join(row) for row in display_board]))
        print()

In [14]:
# Minimax method
def minimax_score(state: TicTacToe, depth: int, player: int) -> int:
    winner = state.check_winner()
    if winner is not None:
        return winner * player
    
    scores = []
    for move in state.get_available_moves():
        new_state = deepcopy(state)
        new_state.make_move(*move, player)
        scores.append(-minimax_score(new_state, depth + 1, -player))
    
    return max(scores) if scores else 0

def minimax(state: TicTacToe, player: int) -> Tuple[int, int]:
    best_score = float('-inf')
    best_move = None
    
    for move in state.get_available_moves():
        new_state = deepcopy(state)
        new_state.make_move(*move, player)
        score = -minimax_score(new_state, 0, -player)
        
        if score > best_score:
            best_score = score
            best_move = move
    
    return best_move

In [15]:
# Alpha-beta method
def alpha_beta_score(state: TicTacToe, depth: int, alpha: float, beta: float, player: int) -> int:
    winner = state.check_winner()
    if winner is not None:
        return winner * player
    
    value = float('-inf')
    for move in state.get_available_moves():
        new_state = deepcopy(state)
        new_state.make_move(*move, player)
        value = max(value, -alpha_beta_score(new_state, depth + 1, -beta, -alpha, -player))
        alpha = max(alpha, value)
        if alpha >= beta:
            break
    
    return value if value != float('-inf') else 0

def alpha_beta(state: TicTacToe, player: int) -> Tuple[int, int]:
    best_score = float('-inf')
    best_move = None
    alpha = float('-inf')
    beta = float('inf')
    
    for move in state.get_available_moves():
        new_state = deepcopy(state)
        new_state.make_move(*move, player)
        score = -alpha_beta_score(new_state, 0, -beta, -alpha, -player)
        
        if score > best_score:
            best_score = score
            best_move = move
        alpha = max(alpha, best_score)
    
    return best_move

In [16]:
# evaluatiob based method
def evaluate_board(state: TicTacToe, player: int) -> float:
    score = 0
    
    # Check rows and columns
    for i in range(3):
        row = state.board[i, :]
        col = state.board[:, i]
        
        # Row evaluation
        if sum(row == player) == 2 and sum(row == 0) == 1:
            score += 1
        if sum(row == -player) == 2 and sum(row == 0) == 1:
            score -= 1
            
        # Column evaluation
        if sum(col == player) == 2 and sum(col == 0) == 1:
            score += 1
        if sum(col == -player) == 2 and sum(col == 0) == 1:
            score -= 1
    
    # Diagonal evaluation
    diag = np.diag(state.board)
    anti_diag = np.diag(np.fliplr(state.board))
    
    for d in [diag, anti_diag]:
        if sum(d == player) == 2 and sum(d == 0) == 1:
            score += 1
        if sum(d == -player) == 2 and sum(d == 0) == 1:
            score -= 1
    
    return score

def evaluation_based(state: TicTacToe, player: int) -> Tuple[int, int]:
    best_score = float('-inf')
    best_move = None
    
    for move in state.get_available_moves():
        new_state = deepcopy(state)
        new_state.make_move(*move, player)
        score = evaluate_board(new_state, player)
        
        if score > best_score:
            best_score = score
            best_move = move
    
    return best_move if best_move else random.choice(state.get_available_moves())

In [17]:
# Monte Carlo Tree search method
class MCTSNode:
    def __init__(self, state: TicTacToe, parent=None, move=None):
        self.state = deepcopy(state)
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.untried_moves = state.get_available_moves()

def monte_carlo_tree_search(state: TicTacToe, player: int) -> Tuple[int, int]:
    root = MCTSNode(state)
    for _ in range(1000):  # Number of simulations
        node = root
        simulation_state = deepcopy(state)
        
        # Selection
        while node.untried_moves == [] and node.children != []:
            node = selection_policy(node)
            simulation_state.make_move(*node.move, player if node.parent.children.index(node) % 2 == 0 else -player)
        
        # Expansion
        if node.untried_moves:
            move = random.choice(node.untried_moves)
            simulation_state.make_move(*move, player if len(node.children) % 2 == 0 else -player)
            node = add_child(node, simulation_state, move)
        
        # Simulation
        current_player = -player if len(node.children) % 2 == 0 else player
        while simulation_state.check_winner() is None:
            move = random.choice(simulation_state.get_available_moves())
            simulation_state.make_move(*move, current_player)
            current_player = -current_player
        
        # Backpropagation
        while node:
            node.visits += 1
            if simulation_state.check_winner() == player:
                node.wins += 1
            node = node.parent
    
    return sorted(root.children, key=lambda c: c.visits)[-1].move

def selection_policy(node: MCTSNode) -> MCTSNode:
    return sorted(node.children, key=lambda c: c.wins/c.visits + math.sqrt(2*math.log(node.visits)/c.visits))[-1]

def add_child(node: MCTSNode, state: TicTacToe, move: Tuple[int, int]) -> MCTSNode:
    child = MCTSNode(state, node, move)
    node.untried_moves.remove(move)
    node.children.append(child)
    return child

In [18]:
class GameStats:
    def __init__(self):
        self.results = defaultdict(lambda: {
            'wins_1': 0,
            'wins_2': 0,
            'draws': 0,
            'times_1': [],
            'times_2': []
        })

    def record_game(self, algo1, algo2, winner, time1, time2):
        key = f"{algo1.__name__} vs {algo2.__name__}"
        if winner == 1:
            self.results[key]['wins_1'] += 1
        elif winner == -1:
            self.results[key]['wins_2'] += 1
        else:
            self.results[key]['draws'] += 1
        self.results[key]['times_1'].append(time1)
        self.results[key]['times_2'].append(time2)

    def print_results(self):
        results_list = []
        for match, stats in self.results.items():
            algo1, algo2 = match.split(' vs ')
            avg_time1 = sum(stats['times_1']) / len(stats['times_1'])
            avg_time2 = sum(stats['times_2']) / len(stats['times_2'])
            
            results_list.append({
                'Algorithm 1': algo1,
                'Algorithm 2': algo2,
                'Wins (Algo 1)': stats['wins_1'],
                'Wins (Algo 2)': stats['wins_2'],
                'Draws': stats['draws'],
                'Avg Time (Algo 1)': f"{avg_time1:.3f}s",
                'Avg Time (Algo 2)': f"{avg_time2:.3f}s"
            })
        
        df = pd.DataFrame(results_list)
        print("\nResults Table:")
        print(df.to_string(index=False))
        return df

In [19]:
def simulate_game_with_stats(method1, method2):
    game = TicTacToe()
    players = [1, -1]
    methods = {1: method1, -1: method2}
    times = {1: [], -1: []}
    
    while game.check_winner() is None:
        current_player = players[0]
        start_time = time.time()
        move = methods[current_player](game, current_player)
        end_time = time.time()
        times[current_player].append(end_time - start_time)
        
        if move:
            game.make_move(*move, current_player)
        players.reverse()
    
    avg_time1 = sum(times[1]) / len(times[1]) if times[1] else 0
    avg_time2 = sum(times[-1]) / len(times[-1]) if times[-1] else 0
    return game.check_winner(), avg_time1, avg_time2

In [20]:
def run_experiments():
    stats = GameStats()
    algorithms = {
        'minimax': minimax,
        'alpha_beta': alpha_beta,
        'evaluation': evaluation_based,
        'mcts': monte_carlo_tree_search
    }
    
    # Define test cases according to the requirements
    test_cases = [
        ('minimax', 'alpha_beta', 1),
        ('minimax', 'evaluation', 1),
        ('minimax', 'mcts', 30),
        ('evaluation', 'alpha_beta', 1),
        ('alpha_beta', 'mcts', 30),
        ('evaluation', 'mcts', 30)
    ]
    
    for algo1_name, algo2_name, num_games in test_cases:
        print(f"\nRunning {num_games} games: {algo1_name} vs {algo2_name}")
        algo1 = algorithms[algo1_name]
        algo2 = algorithms[algo2_name]
        
        for _ in range(num_games):
            winner, time1, time2 = simulate_game_with_stats(algo1, algo2)
            stats.record_game(algo1, algo2, winner, time1, time2)
            
    return stats

In [21]:
def plot_results(df):
    # Win rates plot
    plt.figure(figsize=(12, 6))
    algorithms = ['minimax', 'alpha_beta', 'evaluation', 'mcts']
    win_rates = {algo: [] for algo in algorithms}
    
    for _, row in df.iterrows():
        algo1 = row['Algorithm 1'].split('.')[-1]
        algo2 = row['Algorithm 2'].split('.')[-1]
        total_games = row['Wins (Algo 1)'] + row['Wins (Algo 2)'] + row['Draws']
        
        if algo1 in algorithms:
            win_rates[algo1].append(row['Wins (Algo 1)'] / total_games)
        if algo2 in algorithms:
            win_rates[algo2].append(row['Wins (Algo 2)'] / total_games)
    
    avg_win_rates = {algo: sum(rates)/len(rates) if rates else 0 
                    for algo, rates in win_rates.items()}
    
    plt.bar(avg_win_rates.keys(), avg_win_rates.values())
    plt.title('Average Win Rates by Algorithm')
    plt.ylabel('Win Rate')
    plt.show()

In [ ]:
if __name__ == "__main__":
    # Example match
    winner = simulate_game_with_stats(minimax, alpha_beta)
    if winner == 1:
        print("Player 1 (Method 1) wins!")
    elif winner == -1:
        print("Player 2 (Method 2) wins!")
    else:
        print("It's a draw!")
    stats = run_experiments()
    results_df = stats.print_results()
    plot_results(results_df)